Implementing **MTL** model for *Affect* and *User Recognition*

In [1]:
#from google.colab import files
import _pickle as pickle

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
import tensorflow as tf

from keras.layers import Input, Dense,Dropout,BatchNormalization,concatenate
from keras.models import Model
from keras import optimizers
from tensorflow.keras.utils import plot_model

2025-07-20 12:27:00.974785: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-20 12:27:01.017533: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


### Model Specification

In [2]:
def MTL_model(input_shapes_arr, num_classes_arr, label_names_arr):
  """   
    Building the architecture of the MTL model

    Parameters:
      input_shapes_arr (list): list of tuples containing the shape of the training set
      num_classes_arr  (list): list of int representing the layer size for the final output of the model,
                               AR task is a binary classification (stress vs no-stress)
                               UR task will need to distinguish between the subjects ID in the WESAD dataset
      label_names_arr  (list): list of str representing the output layer labels

      Both num_classes_arr and label_names_arr need to be sharing the same logical order, if the first element of
      num_classes_arr refers to the AR taks output layer size, then the first element of label_names_arr will also
      need to be the label of the AR task

    Returns:
      keras.engine.functional.Functional model instance
  """
  # List will contain the input layers
  inputs = []
  # Setting the dropout rate for the dropout layers
  dropout_rate = .0
  # Setting the learning rate for the optimizer
  learning_rate_opt = 0.001

  # Creating the two separate input layers for their respective task
  for input_shape in input_shapes_arr:
    x = Input(shape=input_shape)
    inputs.append(x)


  # Adding intermediate dense layers for balancing data dimensionality between the two task
  dense_reshape = Dense(100, activation='relu', name='intermediate_balancer')
  x_intermediate = dense_reshape(inputs[0])
  # Adding a batch normalization layer before the dropout layer
  batch = BatchNormalization() (x_intermediate)

  # Adding intermediate dense layers for balancing data dimensionality between the two task
  dense_reshape_2 = Dense(100, activation='relu', name='intermediate_balancer_2')
  x_intermediate_2 = dense_reshape_2(inputs[1])
  # Adding a batch normalization layer before the dropout layer
  batch_2 = BatchNormalization() (x_intermediate_2)

  # Adding dropout layers with the previously declared rate value
  dropout = Dropout(dropout_rate) (batch)
  dropout_2 = Dropout(dropout_rate) (batch_2)

  # Adding the shared layer to share knowledge between AR and UR
  dense_1 = Dense(20, activation='relu',name='shared_layer')
  # Linking the shared layer to the previously created dropout layers
  x1 = dense_1(dropout)
  x2 =  dense_1(dropout_2)

  # Output layer for AR task
  output_1 = Dense(num_classes_arr[0], activation="softmax",name = label_names_arr[0]) (x1)
  loss_1 = 'sparse_categorical_crossentropy'

  # Output layer for UR task
  output_2 = Dense(num_classes_arr[1], activation="softmax",name = label_names_arr[1]) (x2)
  loss_2 = 'sparse_categorical_crossentropy'

  # Creating the model
  model = Model(inputs=inputs, outputs=[output_1,output_2])
  # Setting the model optimizer
  optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate_opt)

  # Compiling the model
  model.compile(optimizer=optimizer, loss=[loss_1,loss_2], metrics=['accuracy'])

  return model 

Model Visualization

In [3]:
# Instancing an example MTL model for architectural visualization
input_shape_1 = (2063,)
input_shape_2 = (2063,)
input_shapes = [input_shape_1,input_shape_2]
num_classes_arr = [2,18]
label_names_arr= ['AR','UR']

model = MTL_model(input_shapes,num_classes_arr,label_names_arr)
plot_model(model)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


2025-07-20 12:27:01.764873: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-07-20 12:27:01.784190: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-07-20 12:27:01.785185: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-07-20 12:27:01.786377: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA
T

### Pipeline Construction

In [4]:
def load_entire_dataset(window_size):
  """   
    Loading the dataset for the AR task with LOSO experimental setup

    Parameters:
      window_size (int) : window size in seconds, two variants have been developed
                          with 30 and 60 seconds window size support

    Returns:
      DataFrame df      : the dataframe representing the dataset for the specified 
                          window size
  """
  path_to_data = f"Stress_NoStress_dataset_{window_size}_sec_window.csv"
  df = pd.read_csv(path_to_data, index_col=0)

  return df

In [5]:
def load_contextual_datasets(window_size):
  """   
    Loading the dataset for the UR task with Start/End experimental setup

    Parameters:
      window_size (int) : window size in seconds, two variants have been developed
                          with 30 and 60 seconds window size support

    Returns:
      DataFrame df               : dataframe for Stress/No-Stress AR task
      list[DataFrame, DataFrame] : list of dataframe(s), containing the 5 minutes 
                                    Start/End split and the 3 minutes variant
      DataFrame df               : dataframe for the final minutes collected for the
                                   Start/End split procedure
  """
  print(f"Working on dataset with Window Size = {window_size} seconds.")

  # List that will contain the DataFrame for the collection of the initial minutes
  # of the Start/End split. It will be populated as [ 5 minutes Start split ; 3 minutes Start split]
  first_minutes_datasets = []
  first_minutes_datasets.append(pd.read_csv(f"{window_size}_sec_window_data_first_5_min.csv", index_col=0))
  first_minutes_datasets.append(pd.read_csv(f"{window_size}_sec_window_data_first_3_min.csv", index_col=0))

  # Reading the last minutes for the End split part of the Start/End split experimental setup
  unified_last_minutes = pd.read_csv(f"{window_size}_sec_window_data_last.csv", index_col=0)
  
  return load_entire_dataset(window_size), first_minutes_datasets, unified_last_minutes

In [6]:
def get_features_to_normalize():
  """   
    Features that will be normalized, expressed as column names

    Returns:
      list : column names of the features that will be normalized
  """
  features_to_normalize = ['median_p', 'mean_p', 'std_p', 'var_p', 'slope_p',
       'min_p', 'max_p', 'fdmean_p', 'fdstd_p', 'sdmean_p', 'sdstd_p',
       'drange_p', 'peaks_p', 'rise_time_p', 'max_deriv_p', 'amp_p',
       'decay_time_p', 'SCR_width_p', 'auc_p', 'median_t', 'mean_t', 'std_t',
       'var_t', 'slope_t', 'min_t', 'max_t', 'fdmean_t', 'fdstd_t', 'sdmean_t',
       'sdstd_t', 'drange_t', 'median_f', 'mean_f', 'std_f', 'var_f',
       'slope_f', 'min_f', 'max_f', 'fdmean_f', 'fdstd_f', 'sdmean_f',
       'sdstd_f', 'drange_f', 'peaks_f', 'rise_time_f', 'max_deriv_f', 'amp_f',
       'decay_time_f', 'SCR_width_f', 'auc_f', 'bpm', 'ibi', 'sdnn', 'sdsd',
       'rmssd', 'pnn20', 'pnn50', 'hr_mad', 'sd1', 'sd2', 's', 'sd1/sd2',
       'breathingrate', 'lf', 'hf', 'lf/hf', 'median_bvp', 'mean_bvp',
       'std_bvp', 'var_bvp', 'min_bvp', 'max_bvp', 'fdmean_bvp', 'fdstd_bvp',
       'sdmean_bvp', 'sdstd_bvp', 'drange_bvp']
       
  return features_to_normalize

In [7]:
def normalizing_data(df):
  """   
    Min-Max Normalization procedure

    Parameters:
      df (DataFrame)    : dataframe on which to perform the Min-Max normalization

    Returns:
      DataFrame df_norm : normalized dataframe
  """
  # Retrieve features that need to be normalized
  # as column items
  features_to_normalize = get_features_to_normalize()
  # Duplicate the dataframe received
  df_norm = df.copy()
 
  # Loop through the features that need to be normalized
  for feature in features_to_normalize:
    # Take the values
    values = df_norm[feature].copy()
    # Calculate the 1-st percentile
    minn = values.quantile(0.01)
    # Calculate the 99-th percentile
    maxx = values.quantile(0.99)
 
    # Winsorization, limiting extreme values for Min-Max
    values[values > maxx] = maxx
    values[values < minn] = minn
 
    # Avoid devision with 0
    if maxx != minn: 
      values = (values-minn)/(maxx-minn)
    else:
      values=0
    # Replace the original features values with the normalized ones  
    df_norm[feature] = values
 
  return df_norm

#### AR Task

In [8]:
from sklearn.metrics import accuracy_score
import numpy as np
def run_AR_task(df_norm, debug):
  eval_arr_task_1 = []
  from MLPLogistic import MLPLogistic
  accs = []
  for sub in df_norm.subjectID.unique():

    print('====== Test subject:', sub)

    # Training subset
    train_df = df_norm[df_norm.subjectID!=sub]
    # Test subset, testing the generalization of the model
    # on previously unseen subject data
    test_df = df_norm[df_norm.subjectID==sub]

    # Retrieve list of features that need to be normalized
    features_to_normalize = get_features_to_normalize()

    # Stress Recognition task
    train_X_1 = train_df[features_to_normalize].values
    # Labels will be the stress/no-stress labels
    train_y_1 = train_df['Label'].values
    model = MLPLogistic(train_X_1.shape[1], 32, 3000, 5e-2)
    model.fit(train_X_1,train_y_1)

    test_X_1 = test_df[features_to_normalize].values
    test_y_1 = test_df['Label'].values

    proba = model.forward(test_X_1)
    pred = np.where(proba>=0.5,1,0)
    accs.append(accuracy_score(test_y_1,pred))

    # User Recognition task
    train_X_2 = train_df[features_to_normalize].values
    # In this case the labels will be the IDs of the subjects
    train_y_2 = train_df['subjectID'].values

    test_X_2 = test_df[features_to_normalize].values
    test_y_2 = test_df['subjectID'].values

    # Preparing the model input parameters
    input_shape_1 = (train_X_1.shape[1],)
    input_shape_2 = (train_X_2.shape[1],)
    input_shapes = [input_shape_1,input_shape_2]
    num_classes_arr = [2,18]                 
    label_names_arr= ['Stress','UID']

    # Creating the input arrays for both training and testing
    train_X = [train_X_1,train_X_2]
    train_y = [train_y_1,train_y_2]
    test_X = [test_X_1,test_X_2]
    test_y = [test_y_1,test_y_2]
    # Unify the test arrays into a single validation tuple for model fitting
    validation_data = (test_X,test_y)

    # Instancing the model
    model = MTL_model(input_shapes,num_classes_arr,label_names_arr)

    # Fit the model with batch size of 15 and fixed epochs of 50
    h = model.fit(train_X,train_y,
                  shuffle=True,
                  batch_size=15,
                  validation_data = validation_data,
                  epochs=50, verbose=debug)
    
    # Appending learning history and evaluation figures
    eval_arr_task_1.append(model.evaluate(test_X,test_y))

  return accs, eval_arr_task_1

#### UR Task

#### Pipeline Execution

In [9]:
def run_pipeline(window_size, debug=0):
  """   
    Entire MTL model pipeline execution

    Parameters:
      window_size (int) : window size in seconds, two variants have been developed
                          with 30 and 60 seconds window size support
      debug (int)       : model fit verbosity, 0 = silent, 1 = progress bar

    Returns:
      model learning history, evaluation results for AR and UR
  """
  # Loading the datasets for the specified window_size variant
  df, first_minutes, unified_last_minutes = load_contextual_datasets(window_size)
  # Min-Max Normalization procedure
  df_norm = normalizing_data(df)

  print("Running AR task")
  # Executing the AR task with LOSO experimental setup
  return run_AR_task(df_norm, debug)

### Plotting Model Results

### Launch Pipeline

In [12]:
# Show all the pipelines output with 1, 0 otherwise
DEBUG = 0
# 60 seconds window size variant
ours_60,ours_30,mtl_60,mtl_30 = [],[],[],[]
for i in range(5) :
    a1_60, a2_60 = run_pipeline(60, debug=0)
    ours_60.append(a1_60)
    mtl_60.append(a2_60)
#a1_60, a2_60 = run_pipeline(60, debug=0)
# 30 seconds window size variant
    a1_30, a2_30 = run_pipeline(30, debug=0)
    ours_30.append(a1_30)
    mtl_30.append(a2_30)

Working on dataset with Window Size = 60 seconds.
Running AR task
====== Test subject: 5


2025-07-20 12:27:04.737382: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2025-07-20 12:27:04.748828: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7f3035d8b6d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-07-20 12:27:04.748849: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 4090, Compute Capability 8.9
2025-07-20 12:27:04.753132: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-07-20 12:27:04.813764: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


5/5 [==============================] - 0s 2ms/step - loss: 20.9001 - Stress_loss: 0.0530 - UID_loss: 20.8470 - Stress_accuracy: 0.9865 - UID_accuracy: 0.0000e+00
====== Test subject: 2
5/5 [==============================] - 0s 1ms/step - loss: 23.4787 - Stress_loss: 0.2163 - UID_loss: 23.2624 - Stress_accuracy: 0.9574 - UID_accuracy: 0.0000e+00
====== Test subject: 3
5/5 [==============================] - 0s 2ms/step - loss: 26.4016 - Stress_loss: 0.5272 - UID_loss: 25.8745 - Stress_accuracy: 0.8472 - UID_accuracy: 0.0000e+00
====== Test subject: 4
5/5 [==============================] - 0s 2ms/step - loss: 26.6872 - Stress_loss: 0.0028 - UID_loss: 26.6844 - Stress_accuracy: 1.0000 - UID_accuracy: 0.0000e+00
====== Test subject: 17
5/5 [==============================] - 0s 2ms/step - loss: 21.0098 - Stress_loss: 1.1493 - UID_loss: 19.8605 - Stress_accuracy: 0.6733 - UID_accuracy: 0.0000e+00
====== Test subject: 10
5/5 [==============================] - 0s 2ms/step - loss: 21.4454 - Stre

In [13]:
mtl_60 = [[d2[-2] for d2 in data] for data in mtl_60] 
mtl_30 = [[d2[-2]for d2 in data] for data in mtl_30]

In [18]:
print(np.mean(ours_60,axis=1))

[0.92622445 0.92484191 0.92576803 0.92260759 0.92212873]


In [21]:
print(f'60 window | ours : {np.mean(ours_60):.4f} | {np.std(np.mean(ours_60,axis=1)):.4f}, MTL : {np.mean(mtl_60):.4f} | {np.std(np.mean(mtl_60,axis=1)):.4f}, acc_diff = {np.mean(ours_60) - np.mean(mtl_60):.4f}')
print(f'30 window | ours : {np.mean(ours_30):.4f} | {np.std(np.mean(ours_30,axis=1)):.4f}, MTL : {np.mean(mtl_30):.4f} | {np.std(np.mean(mtl_30,axis=1)):.4f}, acc_diff = {np.mean(ours_30) - np.mean(mtl_30):.4f}')

60 window | ours : 0.9243 | 0.0017, MTL : 0.9211 | 0.0051, acc_diff = 0.0032
30 window | ours : 0.9221 | 0.0010, MTL : 0.9060 | 0.0065, acc_diff = 0.0161
